In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from tifffile import memmap

from caImageAnalysis import BrukerFish, VolumeFish
from caImageAnalysis.caiman_process import caiman_mcorr
from caImageAnalysis.mesm import *
from caImageAnalysis.visualize import visualize_images, visualize_volumes

pd.options.display.max_colwidth = 120

### Batch analysis

In [2]:
# Change this path to the experiment folder
# exp_folders = ['elavl3H2BGCaMP8m_6dpf_DOI_50ugml_31_20240617', 'elavl3H2BGCaMP8m_9dpf_DOI_50ugml_32_20240620', 'elavl3H2BGCaMP8m_10dpf_DOI_50ugml_33_20240621', 'elavl3H2BGCaMP8m_7dpf_DOI_50ugml_34_20240723',
# 			   'elavl3H2BGCaMP8m_7dpf_DOI_50ugml_35_20240723', 'elavl3H2BGCaMP8m_7dpf_DOI_50ugml_36_20240723', 'elavl3H2BGCaMP8m_8dpf_DOI_50ugml_37_20240724',
# 			   'elavl3H2BGCaMP8m_6dpf_DOI_50ugml_38_20240729', 'elavl3H2BGCaMP8m_6dpf_DOI_50ugml_39_20240729', 'elavl3H2BGCaMP8m_8dpf_DOI_50ugml_40_20240807',
# 			   'elavl3H2BGCaMP8m_8dpf_DOI_50ugml_41_20240807']

exp_folders = ['elavl3H2BGCaMP8m_6dpf_DOI_50ugml_42_20240916', 'elavl3H2BGCaMP8m_6dpf_DOI_50ugml_43_20240923', 'elavl3H2BGCaMP8m_6dpf_DOI_50ugml_44_20240923',
			   'elavl3H2BGCaMP8m_9dpf_DOI_50ugml_45_20240926', 'elavl3H2BGCaMP8m_9dpf_DOI_50ugml_46_20240926']


for exp_f in exp_folders:
	exp_folder = rf'/Volumes/DOI/Bruker/{exp_f}'
	region = 'planes'
	gavage = False
	anatomy = 'anatomystack'

	# Initialize BrukerFish
	fish = BrukerFish(exp_folder, region=region, gavage=gavage, anatomy=anatomy)

	plane = 0

	# Set combination of values to try
	if 'vagal' in fish.data_paths['raw'].name:
		strides = [100]  # start a new patch for pw-rigid motion correction every x pixels
		overlaps = [50]  # overlap between patches (size of patch strides+overlaps)
		max_shifts = [30]  # maximum allowed rigid shifts (in pixels)
		max_deviation_rigid = [10]  # maximum shifts deviation allowed for patch with respect to rigid shifts

	elif 'DOI' in str(fish.data_paths['raw']):  # for zoomed out 1024x1024 DOI recordings
		strides = [160]
		overlaps = [60]
		max_shifts = [40]
		max_deviation_rigid = [40]

	else:  # for hindbrain and other brain regions
		strides = [80]
		overlaps = [30]
		max_shifts = [20]
		max_deviation_rigid = [20]

	# Add the combination of parameters to a mes_df dataframe
	img_path = fish.data_paths['volumes'][str(plane)]['image']
	mes_df = add_mcorr(fish, [img_path], grid=True, 
							strides=strides, 
							overlaps=overlaps,
							max_shifts=max_shifts,
							max_deviation_rigid=max_deviation_rigid)

	mes_df = load_mesmerize(fish)
	run_mesmerize(mes_df)
	mes_df = mes_df.caiman.reload_from_disk()

	# See the rows that were successfully run
	for output in mes_df.loc[:, 'outputs']:
		print(output['success'])

	# Adds every other plane in the volume with the same parameters as the leftover, chosen index
	fish.process_mesmerize_filestructure()
	mes_df = add_volume(fish, algo='mcorr')  

	# Run mesmerize on the entire volume
	run_mesmerize(mes_df)
	mes_df = mes_df.caiman.reload_from_disk()

	# See the rows that were successfully run
	for output in mes_df.loc[:, 'outputs']:
		print(output['success'])

/Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/bruker.py:202: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 202 of the file /Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/bruker.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  Bs_data = BeautifulSoup(log)
/Users/minelarinel/mambaforge/envs/mescore/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML pars

Batch does not exist. Creating batch.pickle
Batch exists. Loading batch.pickle
Running b910ef14-8c0c-4282-94fa-f5a3bcac6e4e with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/DOI/Bruker/elavl3H2BGCaMP8m_6dpf_DOI_50ugml_42_20240916/mesmerize-batch/b910ef14-8c0c-4282-94fa-f5a3bcac6e4e/b910ef14-8c0c-4282-94fa-f5a3bcac6e4e-image_els__d1_640_d2_1024_d3_1_order_F_frames_5142.mmap
finished computing correlation image
True
Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle
Running 4cf99655-d47e-4093-aed5-a164cc9fe082 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/DOI/Bruker/elavl3H2BGCaMP8m_6dpf_DOI_50ugml_42_20240916/mesmerize-batch/4cf99655-d47e-4093-aed5-a164cc9fe082/4cf99655-d47e-4093-aed5-a164cc9fe082-image_els__d1_640_d2_1024_d3_1_order_F_frames_5142.mmap
finished computing correlation image
Running 3ef5dacb-a712-443

python(15430) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


mc finished successfully!
computing projections
Decode mmap filename /Volumes/DOI/Bruker/elavl3H2BGCaMP8m_6dpf_DOI_50ugml_42_20240916/mesmerize-batch/91e1d83c-15fd-4d0e-82c3-de9f5bcbdda5/91e1d83c-15fd-4d0e-82c3-de9f5bcbdda5-image_els__d1_640_d2_1024_d3_1_order_F_frames_5142.mmap
finished computing correlation image
Running aa35c1bf-533f-459b-8941-d6dff0d43de5 with local backend
starting mc


python(35433) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

mc finished successfully!
computing projections
Decode mmap filename /Volumes/DOI/Bruker/elavl3H2BGCaMP8m_9dpf_DOI_50ugml_46_20240926/mesmerize-batch/d662a650-88c7-4b3b-a43c-dcde990a45a2/d662a650-88c7-4b3b-a43c-dcde990a45a2-image_els__d1_640_d2_1024_d3_1_order_F_frames_5142.mmap
finished computing correlation image
Running e8fa0085-a224-4d7f-a225-47ba6dcfcfea with local backend
starting mc


python(29158) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


mc finished successfully!
computing projections
Decode mmap filename /Volumes/DOI/Bruker/elavl3H2BGCaMP8m_9dpf_DOI_50ugml_46_20240926/mesmerize-batch/e8fa0085-a224-4d7f-a225-47ba6dcfcfea/e8fa0085-a224-4d7f-a225-47ba6dcfcfea-image_els__d1_640_d2_1024_d3_1_order_F_frames_5142.mmap
finished computing correlation image
Running 78db67bb-e859-4c41-8e68-213c413086bb with local backend
starting mc


python(42984) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


mc finished successfully!
computing projections
Decode mmap filename /Volumes/DOI/Bruker/elavl3H2BGCaMP8m_9dpf_DOI_50ugml_46_20240926/mesmerize-batch/78db67bb-e859-4c41-8e68-213c413086bb/78db67bb-e859-4c41-8e68-213c413086bb-image_els__d1_640_d2_1024_d3_1_order_F_frames_5142.mmap
finished computing correlation image
True
True
True
True
True


### Change the exp_folder to the path of the experiment

In [53]:
# Change this path to the experiment folder
exp_folder = r'/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528'
region = 'hindbrain'
gavage = True
anatomy = ''

In [54]:
# Initialize BrukerFish
fish = BrukerFish(exp_folder, region=region, gavage=gavage, anatomy=anatomy)
fish.data_paths

/Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/bruker.py:202: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 202 of the file /Users/minelarinel/caImageAnalysis/caImageAnalysis/bruker_2P/bruker.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  Bs_data = BeautifulSoup(log)
/Users/minelarinel/mambaforge/envs/mescore/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML pars

{'raw': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/hindbrain-000'),
 'frametimes': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/hindbrain_frametimes.txt'),
 'raw_image': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/hindbrain_ch2.tif'),
 'rotated': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/raw_rotated.tif'),
 'anatomy': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/anatomy.tif'),
 'log': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/hindbrain-000/hindbrain-000.xml'),
 'voltage_output': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/hindbrain-000/hindbrain-000_Cycle00001_VoltageOutput_001.xml'),
 'references': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_202

## IF VOLUMETRIC: Pick a plane to apply movement correction

In [22]:
iw = visualize_volumes(fish)
iw.show()

RFBOutputContext()

In [23]:
iw.plot.canvas.close()

In [6]:
# Change this to the plane you want to test motion correction parameters on
plane = 0

## Set parameters

In [5]:
# motion correction parameters
strides = (80, 80)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (30, 30)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (20, 20)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 20     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

In [6]:
opts_dict = {'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid}

## Motion correct

In [7]:
if fish.volumetric:
    mcorr_img = caiman_mcorr(fish, plane=plane, **opts_dict)
else:
    mcorr_img = caiman_mcorr(fish, **opts_dict)

  0%|          | 0/1 [00:00<?, ?it/s]

Decode mmap filename /Users/minelarinel/Documents/Duke University/Naumann Lab/DOI/Brain imaging/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000_ch2_els__d1_1024_d2_1024_d3_1_order_F_frames_2046.mmap


100%|██████████| 1/1 [01:00<00:00, 60.75s/it]


Decode mmap filename /Users/minelarinel/Documents/Duke University/Naumann Lab/DOI/Brain imaging/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/motor_pretectum-000_ch2_els__d1_1024_d2_1024_d3_1_order_F_frames_2046.mmap
/Users/minelarinel/Documents/Duke University/Naumann Lab/DOI/Brain imaging/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/memmap_d1_1024_d2_1024_d3_1_order_C_frames_2046.mmap
Decode mmap filename /Users/minelarinel/Documents/Duke University/Naumann Lab/DOI/Brain imaging/elavl3H2BGCaMP6s_11dpf_DOI_50ugml_5_20231104/memmap_d1_1024_d2_1024_d3_1_order_C_frames_2046.mmap


In [ ]:
# Compare the results
if fish.volumetric:
    orig = memmap(fish.data_paths['volumes'][str(plane)]['image'])
else:
    orig = memmap(fish.data_paths['raw_image'])
caiman_iw = visualize_images([orig, mcorr_img], names=['original', 'mcorr'])
caiman_iw.show()

RFBOutputContext()

     4159376 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159378 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159379 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159381 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159381 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159382 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159384 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159386 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     4159388 [base_comm.py:            get_comm():212] [14050] No such comm: fae746f620d04415b0cd37220fe929d9
     41593

In [9]:
caiman_iw.plot.canvas.close()

## Optimize parameters with Mesmerize

### Motion correction parameters

In [55]:
# Set combination of values to try
if 'vagal' in fish.data_paths['raw'].name:
	strides = [100]  # start a new patch for pw-rigid motion correction every x pixels
	overlaps = [50]  # overlap between patches (size of patch strides+overlaps)
	max_shifts = [30]  # maximum allowed rigid shifts (in pixels)
	max_deviation_rigid = [10]  # maximum shifts deviation allowed for patch with respect to rigid shifts

elif 'DOI' in str(fish.data_paths['raw']):  # for zoomed out 1024x1024 DOI recordings
	strides = [160]
	overlaps = [60]
	max_shifts = [40]
	max_deviation_rigid = [40]

else:  # for hindbrain and other brain regions
	strides = [80]
	overlaps = [30]
	max_shifts = [20]
	max_deviation_rigid = [20]

In [56]:
# Add the combination of parameters to a mes_df dataframe
img_path = fish.data_paths['volumes'][str(plane)]['image']
mes_df = add_mcorr(fish, [img_path], grid=True, 
                         strides=strides, 
                         overlaps=overlaps,
                         max_shifts=max_shifts,
                         max_deviation_rigid=max_deviation_rigid)

Batch does not exist. Creating batch.pickle


In [57]:
mes_df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_0,img_stack_0/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2024-08-16T17:16:43,None,None,None,c857fdda-a3a8-4912-89db-33ca1a7dbdda


In [58]:
# See the unique parameters between rows with the same item_name
diffs = mes_df.caiman.get_params_diffs(algo="mcorr", item_name=mes_df.iloc[0]["item_name"])
diffs

/var/folders/48/8y2gqcvx5q5575kwkzngl6qw0000gn/T/ipykernel_14569/688498701.py:2: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  diffs = mes_df.caiman.get_params_diffs(algo="mcorr", item_name=mes_df.iloc[0]["item_name"])


0    {}
Name: params, dtype: object

### Run motion correction with the given parameter space

In [59]:
mes_df = load_mesmerize(fish)
run_mesmerize(mes_df)
mes_df = mes_df.caiman.reload_from_disk()

Batch exists. Loading batch.pickle
Running c857fdda-a3a8-4912-89db-33ca1a7dbdda with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/mesmerize-batch/c857fdda-a3a8-4912-89db-33ca1a7dbdda/c857fdda-a3a8-4912-89db-33ca1a7dbdda-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
finished computing correlation image


In [48]:
# See the rows that were successfully run
for output in mes_df.loc[:, 'outputs']:
    print(output['success'])

True


### Visualize outputs

In [27]:
# Compare the raw image to motion corrected ones
mes_df = load_mesmerize(fish)
mcorr_iw = visualize_mesmerize(mes_df, 'mcorr')
mcorr_iw.show()

Batch exists. Loading batch.pickle
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_10dpf_fed_glucose_500mM_3_20230818/mesmerize-batch/6dbcf444-a022-49f2-9564-b1be818fd7a3/6dbcf444-a022-49f2-9564-b1be818fd7a3-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap


RFBOutputContext()

In [28]:
mcorr_iw.plot.canvas.close()

In [21]:
# Visualize the difference of each frame from the mean
mcorr_diff_iw = visualize_diff(mes_df)
mcorr_diff_iw.show()

Decode mmap filename /Volumes/DOI/Bruker/elavl3H2BGCaMP8m_10dpf_DOI_50ugml_22_20240426/mesmerize-batch/492b1576-89f8-495b-a8a5-89ba283d13e0/492b1576-89f8-495b-a8a5-89ba283d13e0-image_els__d1_730_d2_1024_d3_1_order_F_frames_4565.mmap


RFBOutputContext()

In [22]:
mcorr_diff_iw.plot.canvas.close()

### Choose an index with the best parameters

In [30]:
# Change the row index here that you want to keep
# If you want to delete all rows to start over, keep the list empty
indices = [0]

In [31]:
# Removes every row except for the chosen index in the indices list
df = load_mesmerize(fish)
mes_df = clean_mesmerize(df, indices)  
mes_df

Batch exists. Loading batch.pickle


,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_0,img_stack_0/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",{'mean-projection-path': 6dbcf444-a022-49f2-9564-b1be818fd7a3/6dbcf444-a022-49f2-9564-b1be818fd7a3_mean_projection.n...,2024-08-16T14:26:38,2024-08-16T14:27:34,54.31 sec,None,6dbcf444-a022-49f2-9564-b1be818fd7a3


## Run motion correction on all planes

In [60]:
# Adds every other plane in the volume with the same parameters as the leftover, chosen index
fish.process_mesmerize_filestructure()
mes_df = add_volume(fish, algo='mcorr')  
mes_df

Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle


,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_0,img_stack_0/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",{'mean-projection-path': c857fdda-a3a8-4912-89db-33ca1a7dbdda/c857fdda-a3a8-4912-89db-33ca1a7dbdda_mean_projection.n...,2024-08-16T17:16:43,2024-08-16T17:17:38,52.36 sec,None,c857fdda-a3a8-4912-89db-33ca1a7dbdda
1,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2024-08-16T17:17:38,None,None,None,3c72f957-4645-4d93-8e99-3c2f916edfd4
2,mcorr,img_stack_2,img_stack_2/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2024-08-16T17:17:38,None,None,None,03cec7d2-c86c-439e-885f-11effc1391c0
3,mcorr,img_stack_3,img_stack_3/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2024-08-16T17:17:38,None,None,None,1cc5044e-7ea1-4a5b-9e80-2aa3b053517c
4,mcorr,img_stack_4,img_stack_4/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2024-08-16T17:17:38,None,None,None,f4cb6895-6773-4ee2-a09c-648d1e4076c3


In [61]:
# Run mesmerize on the entire volume
run_mesmerize(mes_df)
mes_df = mes_df.caiman.reload_from_disk()

# See the rows that were successfully run
for output in mes_df.loc[:, 'outputs']:
    print(output['success'])

Running 3c72f957-4645-4d93-8e99-3c2f916edfd4 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/mesmerize-batch/3c72f957-4645-4d93-8e99-3c2f916edfd4/3c72f957-4645-4d93-8e99-3c2f916edfd4-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
finished computing correlation image
Running 03cec7d2-c86c-439e-885f-11effc1391c0 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/mesmerize-batch/03cec7d2-c86c-439e-885f-11effc1391c0/03cec7d2-c86c-439e-885f-11effc1391c0-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
finished computing correlation image
Running 1cc5044e-7ea1-4a5b-9e80-2aa3b053517c with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7

### Visualize outputs

In [62]:
# Visualize the motion corrected output of all planes
mes_df = load_mesmerize(fish)
mcorr_iw = visualize_mesmerize(mes_df, 'mcorr')
mcorr_iw.show()

Batch exists. Loading batch.pickle
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/mesmerize-batch/c857fdda-a3a8-4912-89db-33ca1a7dbdda/c857fdda-a3a8-4912-89db-33ca1a7dbdda-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/mesmerize-batch/3c72f957-4645-4d93-8e99-3c2f916edfd4/3c72f957-4645-4d93-8e99-3c2f916edfd4-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/mesmerize-batch/03cec7d2-c86c-439e-885f-11effc1391c0/03cec7d2-c86c-439e-885f-11effc1391c0-image_els__d1_512_d2_512_d3_1_order_F_frames_1174.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_7dpf_fed_AITC_25mM_52_20240528/mesmerize-batch/1cc5044e-7ea1-4a5b-9e80-2aa3b053517c/1cc5044e-7ea1-4a5b-9e80-2aa3b053517c-image_els__d1_512_d2_512_d3_1_order_F_frames_117

RFBOutputContext()

In [63]:
mcorr_iw.plot.canvas.close()